##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Redes Neurais Recorrentes (RNN) com Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/keras/rnn"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Veja em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/keras/rnn.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/keras/rnn.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/keras/rnn.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

## Introdução

Redes neurais recorrentes (RNN - Recurrent neural networks) são uma classe de redes neurais poderosas para modelar dados sequenciais, tais como séries temporais ou linguagem natural.

Esquematicamente, uma camada RNN usa um loop `for` para iterar sobre os timesteps de uma sequência, enquanto mantém um estado interno que codifica informações sobre os timesteps vistos até o momento.

A API Keras RNN foi projetada com foco em:

- **Facilidade de uso**: as camadas incorporadas `keras.layers.RNN`, `keras.layers.LSTM`, `keras.layers.GRU` permitem que você construa rapidamente modelos recorrentes sem ter que tomar decisões difíceis de configuração.

- **Facilidade de personalização**: Você também pode definir sua própria camada de célula RNN (a parte interna do loop `for` ) com comportamento personalizado e usá-la com a camada `keras.layers.RNN` genérica (o próprio loop `for`). Isto permite que você crie rapidamente protótipos de diferentes ideias de pesquisa de maneira flexível com o mínimo de código.

## Configuração

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Camadas RNN integradas: um exemplo simples

Existem três camadas RNN integradas no Keras:

1. `keras.layers.SimpleRNN`, uma RNN totalmente conectada onde a saída do timestep anterior deve ser alimentada ao próximo timestep.

2. `keras.layers.GRU`, proposta pela primeira vez em [Cho et al., 2014](https://arxiv.org/abs/1406.1078).

3. `keras.layers.LSTM`, proposta pela primeira vez em [Hochreiter &amp; Schmidhuber, 1997](https://www.bioinf.jku.at/publications/older/2604.pdf).

No início de 2015, foram criadas as primeiras implementações Python de código aberto reutilizáveis ​​de LSTM e GRU do Keras.

Aqui está um exemplo simples de um modelo `Sequential` que processa sequências de números inteiros, incorpora cada número inteiro em um vetor de 64 dimensões e, em seguida, processa a sequência de vetores usando uma camada `LSTM`.

In [ ]:
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))

model.summary()

RNNs integrados oferecem suporte a vários recursos úteis:

- Dropout recorrente, por meio dos argumentos `dropout` e `recurrent_dropout`
- Capacidade de processar uma sequência de entrada ao contrário, através do argumento `go_backwards`
- Desenrolamento de loop (que pode causar uma grande aceleração ao processar sequências curtas na CPU), por meio do argumento `unroll`
- ...e mais.

Para obter mais informações, consulte a [documentação da API RNN](https://keras.io/api/layers/recurrent_layers/).

## Saídas e estados

Por padrão, a saída de uma camada RNN contém um único vetor por amostra. Este vetor é a saída da célula RNN correspondente ao último timestep, contendo informações sobre toda a sequência de entrada. O formato desta saída é `(batch_size, units)` onde `units` corresponde ao argumento `units` passado para o construtor da camada.

Uma camada RNN também pode retornar toda a sequência de saídas para cada amostra (um vetor por timestep por amostra), se você definir `return_sequences=True`. O formato dessa saída é `(batch_size, timesteps, units)`.

In [ ]:
model = keras.Sequential()
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(layers.GRU(256, return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(layers.SimpleRNN(128))

model.add(layers.Dense(10))

model.summary()

Além disso, uma camada RNN pode retornar seu(s) estado(s) interno(s) final(ais). Os estados retornados podem ser usados ​​para retomar a execução da RNN posteriormente ou [para inicializar outra RNN](https://arxiv.org/abs/1409.3215). Essa configuração é frequentemente usada no modelo sequence-to-sequence do encoder-decoder, em que o estado final do encoder é usado como o estado inicial do decoder.

Para configurar uma camada RNN para retornar seu estado interno, defina o parâmetro `return_state` como `True` ao criar a camada. Observe que `LSTM` possui 2 tensores de estado, mas `GRU` possui apenas um.

Para configurar o estado inicial da camada, basta chamar a camada com o argumento de palavra-chave adicional `initial_state`. Observe que o formato do estado precisa corresponder ao tamanho da unidade da camada, como no exemplo abaixo.

In [ ]:
encoder_vocab = 1000
decoder_vocab = 2000

encoder_input = layers.Input(shape=(None,))
encoder_embedded = layers.Embedding(input_dim=encoder_vocab, output_dim=64)(
    encoder_input
)

# Return states in addition to output
output, state_h, state_c = layers.LSTM(64, return_state=True, name="encoder")(
    encoder_embedded
)
encoder_state = [state_h, state_c]

decoder_input = layers.Input(shape=(None,))
decoder_embedded = layers.Embedding(input_dim=decoder_vocab, output_dim=64)(
    decoder_input
)

# Pass the 2 states to a new LSTM layer, as initial state
decoder_output = layers.LSTM(64, name="decoder")(
    decoder_embedded, initial_state=encoder_state
)
output = layers.Dense(10)(decoder_output)

model = keras.Model([encoder_input, decoder_input], output)
model.summary()

## Camadas RNN e células RNN

Além das camadas RNN integradas, a API RNN também fornece APIs em nível de célula. Ao contrário das camadas RNN, que processam lotes inteiros de sequências de entrada, a célula RNN processa apenas um único timestep.

A célula é o interior do loop `for` de uma camada RNN. Envolver uma célula dentro de uma camada `keras.layers.RNN` fornece uma camada capaz de processar lotes de sequências, por exemplo `RNN(LSTMCell(10))`.

Matematicamente, `RNN(LSTMCell(10))` produz o mesmo resultado que `LSTM(10)`. Na verdade, a implementação dessa camada no TF v1.x foi apenas criar a célula RNN correspondente e empacotá-la numa camada RNN. No entanto, usar as camadas integradas `GRU` e `LSTM` permite o uso de CuDNN e você pode assim obter um melhor desempenho.

Existem três células RNN integradas, cada uma delas correspondendo à camada RNN correspondente.

- `keras.layers.SimpleRNNCell` corresponde à camada `SimpleRNN`.

- `keras.layers.GRUCell` corresponde à camada `GRU`.

- `keras.layers.LSTMCell` corresponde à camada `LSTM`.

A abstração de célula, juntamente com a classe genérica `keras.layers.RNN`, facilita muito a implementação de arquiteturas RNN customizadas para sua pesquisa.

## Statefulness entre lotes

Ao processar sequências muito longas (possivelmente infinitas), você talvez queira usar o padrão **statefulness entre lotes** (cross-batch statefulness).

Geralmente, o estado interno de uma camada RNN é reiniciado toda vez que ela encontra um novo lote (ou seja, cada amostra vista pela camada é considerada independente do seu passado). A camada só irá preserver um estado enquanto processa uma determinada amostra.

No entanto, se você tiver sequências muito longas, é útil dividi-las em sequências mais curtas e alimentar essas sequências mais curtas sequencialmente numa camada RNN sem reiniciar o estado da camada. Dessa forma, a camada pode reter informações sobre toda a sequência, mesmo que esteja vendo apenas uma subsequência de cada vez.

Você pode fazer isso definindo `stateful=True` no construtor.

Se você tiver uma sequência `s = [t0, t1, ... t1546, t1547]` , você poderia dividi-la da forma a seguir:

```
s1 = [t0, t1, ... t100]
s2 = [t101, ... t201]
...
s16 = [t1501, ... t1547]
```

E depois você poderia processá-la da usando:

```python
lstm_layer = layers.LSTM(64, stateful=True)
for s in sub_sequences:
  output = lstm_layer(s)
```

Quando quiser limpar o estado, você pode usar `layer.reset_states()`.

> Observação: Nesta configuração, a amostra `i` em um determinado lote é considerada a continuação da amostra `i` do lote anterior. Isto significa que todos os lotes devem conter o mesmo número de amostras (tamanho do lote). Por exemplo, se um lote contém `[sequence_A_from_t0_to_t100, sequence_B_from_t0_to_t100]`, o próximo lote deve conter `[sequence_A_from_t101_to_t200, sequence_B_from_t101_to_t200]`.

Aqui está um exemplo completo:

In [ ]:
paragraph1 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph2 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph3 = np.random.random((20, 10, 50)).astype(np.float32)

lstm_layer = layers.LSTM(64, stateful=True)
output = lstm_layer(paragraph1)
output = lstm_layer(paragraph2)
output = lstm_layer(paragraph3)

# reset_states() will reset the cached state to the original initial_state.
# If no initial_state was provided, zero-states will be used by default.
lstm_layer.reset_states()


### Reuso de estado em RNNs

<a id="rnn_state_reuse"></a>

Os estados registrados da camada RNN não são incluídos em `layer.weights()`. Se quiser reutilizar o estado de uma camada RNN, você pode recuperar o valor dos estados por `layer.states` e usá-lo como o estado inicial para uma nova camada através da API funcional Keras como `new_layer(inputs, initial_state=layer.states)`, ou através da subclasse de um modelo.

Observe que o modelo sequencial pode não ser usado neste caso, pois ele suporta apenas camadas com entrada e saída únicas, a entrada adicional do estado inicial faz com que seja impossível usá-lo aqui.

In [ ]:
paragraph1 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph2 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph3 = np.random.random((20, 10, 50)).astype(np.float32)

lstm_layer = layers.LSTM(64, stateful=True)
output = lstm_layer(paragraph1)
output = lstm_layer(paragraph2)

existing_state = lstm_layer.states

new_lstm_layer = layers.LSTM(64)
new_output = new_lstm_layer(paragraph3, initial_state=existing_state)


## RNNs bidirecionais

Para sequências que não sejam séries temporais (por exemplo, texto), é comum que um modelo RNN tenha um desempenho melhor se não apenas processar a sequência do início ao fim, mas também de trás para frente. Por exemplo, para prever a próxima palavra numa frase, geralmente é útil conhecer o contexto em torno da palavra, não apenas as palavras que vêm antes dela.

Keras fornece uma API prática para você construir RNNs bidirecionais desse tipo: o wrapper `keras.layers.Bidirectional`.

In [ ]:
model = keras.Sequential()

model.add(
    layers.Bidirectional(layers.LSTM(64, return_sequences=True), input_shape=(5, 10))
)
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(10))

model.summary()

Nos bastidores, `Bidirectional` copiará a camada RNN passada e inverterá o campo `go_backwards` da camada recém-copiada, para processar as entradas na ordem inversa.

A saída da RNN `Bidirectional` será, por padrão, a concatenação da saída da camada "para frente" e da saída da camada "para trás". Se você precisar de um comportamento de fusão diferente, por exemplo, a concatenação, altere o parâmetro `merge_mode` no construtor do wrapper `Bidirectional`. Para mais detalhes sobre `Bidirectional`, consulte a [documentação da API](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional/).

## Otimização de desempenho e kernels CuDNN

No TensorFlow 2.0, as camadas LSTM e GRU incorporadas foram atualizadas para aproveitar os kernels CuDNN por padrão quando uma GPU está disponível. Com essa alteração, as camadas anteriores `keras.layers.CuDNNLSTM/CuDNNGRU` foram descontinuadas e você pode criar seu modelo sem se preocupar com o hardware em que ele será executado.

Já que o kernel CuDNN foi construído levando em conta determinadas suposições, isto significa que a camada **não poderá usar o kernel CuDNN se você alterar os padrões das camadas LSTM ou GRU integradas**. Por exemplo:

- Alterar a função `activation` de `tanh` para outra coisa.
- Alterar a função `recurrent_activation` de `sigmoid` para outra coisa.
- Usar `recurrent_dropout` &gt; 0.
- Definir `unroll` como True, o que força o LSTM/GRU a decompor o `tf.while_loop` interno em um loop `for` desenrolado.
- Definir `use_bias` como False.
- Usar mascaramento quando os dados de entrada não forem preenchidos estritamente à direita (se a máscara corresponder a dados preenchidos estritamente à direita, o CuDNN ainda pode ser usado. Este é o caso mais comum).

Para obter uma lista detalhada de restrições, consulte a documentação das camadas [LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM/) e [GRU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU/).

### Usando kernels CuDNN quando disponíveis

Vamos construir um modelo LSTM simples para demonstrar a diferença de desempenho.

Usaremos como sequências de entrada a sequência de linhas de dígitos MNIST (tratando cada linha de pixels como um timestep) e faremos a previsão do rótulo do dígito.

In [ ]:
batch_size = 64
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
input_dim = 28

units = 64
output_size = 10  # labels are from 0 to 9

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
    if allow_cudnn_kernel:
        # The LSTM layer with default options uses CuDNN.
        lstm_layer = keras.layers.LSTM(units, input_shape=(None, input_dim))
    else:
        # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
        lstm_layer = keras.layers.RNN(
            keras.layers.LSTMCell(units), input_shape=(None, input_dim)
        )
    model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.BatchNormalization(),
            keras.layers.Dense(output_size),
        ]
    )
    return model


Vamos carregar o dataset MNIST:

In [ ]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
sample, sample_label = x_train[0], y_train[0]

Vamos criar uma instância de modelo e treiná-la.

Escolhemos `sparse_categorical_crossentropy` como a função de perda para o modelo. A saída do modelo tem formato de `[batch_size, 10]`. O alvo para o modelo é um vetor inteiro, cada um dos inteiros está no intervalo de 0 a 9.

In [ ]:
model = build_model(allow_cudnn_kernel=True)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)


model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=1
)

Agora, vamos comparar com um modelo que não usa o kernel CuDNN:

In [ ]:
noncudnn_model = build_model(allow_cudnn_kernel=False)
noncudnn_model.set_weights(model.get_weights())
noncudnn_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)
noncudnn_model.fit(
    x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=1
)

Ao rodar numa máquina com GPU NVIDIA e CuDNN instalados, o modelo construído com CuDNN pode ser treinado com muito mais rapidez em comparação com o modelo que usa o kernel TensorFlow comum.

O mesmo modelo habilitado para CuDNN também pode ser usado para rodar inferências num ambiente CPU-only. A anotação `tf.device` abaixo está apenas forçando o posicionamento do dispositivo. O modelo será executado na CPU por padrão se nenhuma GPU estiver disponível.

Você simplesmente não precisa mais se preocupar com o hardware onde está executando. Não é legal?

In [ ]:
import matplotlib.pyplot as plt

with tf.device("CPU:0"):
    cpu_model = build_model(allow_cudnn_kernel=True)
    cpu_model.set_weights(model.get_weights())
    result = tf.argmax(cpu_model.predict_on_batch(tf.expand_dims(sample, 0)), axis=1)
    print(
        "Predicted result is: %s, target result is: %s" % (result.numpy(), sample_label)
    )
    plt.imshow(sample, cmap=plt.get_cmap("gray"))

## RNNs com entradas list/dict ou entradas aninhadas

Estruturas aninhadas permitem que os implementadores incluam mais informações num único timestep. Por exemplo, um quadro de vídeo pode ter entradas de áudio e vídeo simultâneas. A forma de dados neste caso poderia ser:

`[batch, timestep, {"video": [height, width, channel], "audio": [frequency]}]`

Em outro exemplo, dados de caligrafia poderiam armazenar as coordenadas x e y para a posição atual da caneta, assim como informações de pressão da caneta sobre o papel. Assim, a representação dos dados poderia ser:

`[batch, timestep, {"location": [x, y], "pressure": [force]}]`

O código a seguir mostra um exemplo de como criar uma célula RNN personalizada que aceite essas entradas estruturadas.

### Defina uma célula personalizada que suporte entrada/saída aninhada

Veja [Criando novas camadas e modelos através de subclasses](https://www.tensorflow.org/guide/keras/custom_layers_and_models/) para mais detalhes sobre como escrever suas próprias camadas.

In [ ]:
class NestedCell(keras.layers.Layer):
    def __init__(self, unit_1, unit_2, unit_3, **kwargs):
        self.unit_1 = unit_1
        self.unit_2 = unit_2
        self.unit_3 = unit_3
        self.state_size = [tf.TensorShape([unit_1]), tf.TensorShape([unit_2, unit_3])]
        self.output_size = [tf.TensorShape([unit_1]), tf.TensorShape([unit_2, unit_3])]
        super(NestedCell, self).__init__(**kwargs)

    def build(self, input_shapes):
        # expect input_shape to contain 2 items, [(batch, i1), (batch, i2, i3)]
        i1 = input_shapes[0][1]
        i2 = input_shapes[1][1]
        i3 = input_shapes[1][2]

        self.kernel_1 = self.add_weight(
            shape=(i1, self.unit_1), initializer="uniform", name="kernel_1"
        )
        self.kernel_2_3 = self.add_weight(
            shape=(i2, i3, self.unit_2, self.unit_3),
            initializer="uniform",
            name="kernel_2_3",
        )

    def call(self, inputs, states):
        # inputs should be in [(batch, input_1), (batch, input_2, input_3)]
        # state should be in shape [(batch, unit_1), (batch, unit_2, unit_3)]
        input_1, input_2 = tf.nest.flatten(inputs)
        s1, s2 = states

        output_1 = tf.matmul(input_1, self.kernel_1)
        output_2_3 = tf.einsum("bij,ijkl->bkl", input_2, self.kernel_2_3)
        state_1 = s1 + output_1
        state_2_3 = s2 + output_2_3

        output = (output_1, output_2_3)
        new_states = (state_1, state_2_3)

        return output, new_states

    def get_config(self):
        return {"unit_1": self.unit_1, "unit_2": unit_2, "unit_3": self.unit_3}


### Construa um modelo RNN com entrada/saída aninhada

Vamos construir um modelo Keras que use uma camada `keras.layers.RNN` e a célula personalizada que acabamos de definir.

In [ ]:
unit_1 = 10
unit_2 = 20
unit_3 = 30

i1 = 32
i2 = 64
i3 = 32
batch_size = 64
num_batches = 10
timestep = 50

cell = NestedCell(unit_1, unit_2, unit_3)
rnn = keras.layers.RNN(cell)

input_1 = keras.Input((None, i1))
input_2 = keras.Input((None, i2, i3))

outputs = rnn((input_1, input_2))

model = keras.models.Model([input_1, input_2], outputs)

model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

### Treine o modelo com dados gerados aleatoriamente

Já que não há um bom candidato a dataset para este modelo, usamos dados aleatórios do Numpy para demonstração.

In [ ]:
input_1_data = np.random.random((batch_size * num_batches, timestep, i1))
input_2_data = np.random.random((batch_size * num_batches, timestep, i2, i3))
target_1_data = np.random.random((batch_size * num_batches, unit_1))
target_2_data = np.random.random((batch_size * num_batches, unit_2, unit_3))
input_data = [input_1_data, input_2_data]
target_data = [target_1_data, target_2_data]

model.fit(input_data, target_data, batch_size=batch_size)

Com a camada Keras `keras.layers.RNN` você só precisa definir a lógica matemática para uma etapa individual dentro da sequência, e a camada `keras.layers.RNN` cuidará da iteração da sequência para você. É uma maneira incrivelmente poderosa de prototipar rapidamente novos tipos de RNNs (por exemplo, uma variante LSTM).

Para mais informações, consulte os [documentos da API](https://https://www.tensorflow.org/api_docs/python/tf/keras/layers/RNN/).